# 穴掘り法(迷路生成アルゴリズム)

ここで生成される迷路とは、2 次元配列に 0 と 1 で構成されます。1 が通れない壁部分で 0 が通れる通路部分を意味します。

```text
1111111
1000001
1010111
1010001
1011101
1010001
1111111
```

迷路の成立条件として、以下の二つがある

1. 外の通路につながらない死に領域(閉じた領域)がない
2. 通路がループにならない

# 穴掘り法の手順

1. 迷路全体を構成する2次元配列を、幅高さ5以上の奇数で生成します。
2. 迷路の外周を通路とし、それ以外を壁とします。
3. x, yともに奇数となる座標(任意)を穴掘り開始座標とします。
4. 穴掘りを開始します。
    - 指定座標を通路とします。
    - 次に掘り進める方向(1セル先が通路かつ2セル先が壁の方向)をランダムで決定し、2セル先まで通路とします。  
    掘り進められなくなるまで繰り返します。
    - 掘り進めた結果四方のどこにも進めなくなった場合、すでに通路となった座標(x, yともに奇数)をランダムに取得し、  
    の穴掘り処理を再帰的に呼び出します。
5. 外周を壁に戻します。


In [9]:
import random
from enum import Enum
from typing import List
from collections import deque


class Direction(Enum):
    Up = 0,
    Right = 1,
    Down = 2,
    Left = 3


class Cell:

    def __init__(self, x: int, y: int) -> None:
        self.X = x
        self.Y = y

def maze_generator_dig(size):
    path = "_"
    wall = "■"

    # サイズが5未満では迷路を生成できない(壁と繋がってしまうため)
    if size < 5:
        size = 5
    if size % 2 == 0:
        size += 1

    maze = [[""] * (size) for _ in range(size)]
    # 穴掘りの開始地点候補。この配列の長さを見て掘りを終了する
    start_cells: List[Cell] = []

    # 迷路の枠の作成(穴を掘るため、外周を通路、それ以外を壁とい初期状態を作る)
    for row in range(size):
        for col in range(size):
            if row == 0 or col == 0 or row == size - 1 or col == size - 1:
                maze[row][col] = path
            else:
                maze[row][col] = wall

    def set_path(x, y):
        maze[x][y] = path
        if x % 2 == 1 and y % 2 == 1:
            # 一度pathになったcellからしかstartできないので。set_pathのタイミングでstart_cellsを増やしていく。
            start_cells.append(Cell(x, y))

    def get_start_cell():
        if len(start_cells) == 0:
            return None
        start_index = random.randint(0, len(start_cells)-1)
        cell = start_cells[start_index]
        del start_cells[start_index]
        return cell

    def dig(x, y):
        while True:
            # 掘り進めることができる方向を保持
            directions = []
            if maze[x][y - 1] == wall and maze[x][y - 2] == wall:
                directions.append(Direction.Up)
            if maze[x + 1][y] == wall and maze[x + 2][y] == wall:
                directions.append(Direction.Right)
            if maze[x][y + 1] == wall and maze[x][y + 2] == wall:
                directions.append(Direction.Down)
            if maze[x - 1][y] == wall and maze[x - 2][y] == wall:
                directions.append(Direction.Left)

            if len(directions) == 0: break

            set_path(x, y)

            direction_index = random.randint(0, len(directions)-1)
            if directions[direction_index] == Direction.Up:
                y -= 1
                set_path(x, y)
                y -= 1
                set_path(x, y)
            elif directions[direction_index] == Direction.Right:
                x += 1
                set_path(x, y)
                x += 1
                set_path(x, y)
            elif directions[direction_index] == Direction.Down:
                y += 1
                set_path(x, y)
                y += 1
                set_path(x, y)
            elif directions[direction_index] == Direction.Left:
                x -= 1
                set_path(x, y)
                x -= 1
                set_path(x, y)

        next_cell = get_start_cell()
        if next_cell:
            dig(next_cell.X, next_cell.Y)

        

    dig(1,1)


    # 外枠を壁に戻す
    for row in range(size):
        for col in range(size):
            if row == 0 or col == 0 or row == size - 1 or col == size - 1:
                maze[row][col] = wall

    # 迷路の出力
    for row in maze:
        print(*row)


maze_generator_dig(20)

■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
■ _ _ _ ■ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ■
■ ■ ■ _ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ _ ■ _ ■ _ ■
■ _ ■ _ _ _ _ _ _ _ _ _ _ _ _ _ ■ _ ■ _ ■
■ _ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ _ ■
■ _ _ _ ■ _ _ _ _ _ ■ _ _ _ ■ _ _ _ _ _ ■
■ _ ■ ■ ■ _ ■ ■ ■ _ ■ _ ■ _ ■ _ ■ ■ ■ _ ■
■ _ _ _ _ _ ■ _ ■ _ _ _ ■ _ _ _ ■ _ _ _ ■
■ _ ■ ■ ■ ■ ■ _ ■ _ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
■ _ ■ _ _ _ _ _ ■ _ _ _ _ _ _ _ _ _ _ _ ■
■ _ ■ _ ■ ■ ■ ■ ■ _ ■ ■ ■ _ ■ ■ ■ ■ ■ _ ■
■ _ ■ _ _ _ ■ _ ■ _ ■ _ ■ _ _ _ _ _ ■ _ ■
■ _ ■ ■ ■ _ ■ _ ■ _ ■ _ ■ ■ ■ ■ ■ _ ■ ■ ■
■ _ ■ _ _ _ ■ _ _ _ ■ _ ■ _ _ _ ■ _ _ _ ■
■ _ ■ _ ■ ■ ■ ■ ■ ■ ■ _ ■ _ ■ ■ ■ ■ ■ _ ■
■ _ _ _ _ _ ■ _ _ _ _ _ ■ _ _ _ ■ _ _ _ ■
■ ■ ■ ■ ■ _ ■ _ ■ _ ■ ■ ■ ■ ■ _ ■ _ ■ _ ■
■ _ _ _ ■ _ _ _ ■ _ ■ _ _ _ ■ _ ■ _ ■ _ ■
■ _ ■ _ ■ _ ■ ■ ■ _ ■ _ ■ _ ■ _ ■ _ ■ _ ■
■ _ ■ _ _ _ ■ _ _ _ ■ _ ■ _ _ _ _ _ ■ _ ■
■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
